## Predicting fractures by applying neural networks to conventional well-logging data  
Joshua Poirier  
Geoscientist  
NEOS  
April 2017  

### Abstract  

This case study comes from Guangren Shi's chapter on Artificial Neural Networks from the book "Data Mining and Knowledge Discovery for Geoscientists." You can
purchase the book from [Amazon](https://www.amazon.com/Data-Mining-Knowledge-Discovery-Geoscientists/dp/0124104371/ref=sr_1_1?ie=UTF8&qid=1490908644&sr=8-1&keywords=data+mining+and+knowledge+discovery+for+geoscientists+%2B+guangren+shi) or directly from the publisher, [Elsevier](https://www.elsevier.com/books/data-mining-and-knowledge-discovery-for-geoscientists/shi/978-0-12-410437-2). The objective is to predict fractures using conventional well-logging data. This data has practical value when the data of the imaging log and core samples are limited.  

Shi describes the scenario as follows:  

> Located southeast of the Biyang Sag in Nanxiang Basin in central China, the Anpeng Oil-field covers an area of about 17.5 square kilometers, close to Tanghe-zaoyuan in the northwest-west, striking a large boundary fault in the south, and close to a deep sag in the east. As an inherited nose structure plunging from northwest to southeast, this oilfield is a simple structure without faults, where commercial oil and gas flows have been discovered (Ming et al., 2005; Wang et al., 2006). One of its favorable pool-forming conditions is that the fractures are found to be well developed at formations as deep as 2800 m or more. These fractures provide favorable oil-gas migration pathways and enlarged the accumulation space.

Computationally, instead of writing the neural network code from scratch I'll be using TFLearn, a high level library built on top of TensorFlow to build neural networks. TensorFlow was developed by Google, and is open-source (free!).  

### Introduction  

The data was transcribed from Shi's book and includes data from 33 samples in Wells An1 and An2, of which he used 29 as learning samples; holding out 4 as a test set. The data features available are summarized below. Units are not given as each log has been normalized over the interval [0, 1].  

| Variable name | Description                                                  |
| ------------- | ------------------------------------------------------------ |
| Sample        | Sample number                                                |
| Well          | Well number                                                  |
| Depth         | Measured depth in meters                                     |
| DT            | Acoustic time                                                |
| RHO           | Compensated neutron density                                  |
| PHIN          | Compensated neutron porosity                                 |
| R_XO          | Microspherically focused resistivity                         |
| R_LLD         | Deep laterolog resistivity                                   |
| R_LLS         | Shallow laterolog resistivity                                |
| R_DS          | Absolute difference between R_LLD and R_LLS                  |
| IL            | Fracture identification determined by imaging log (1=fracture, 2=nonfracture) |

I'll get started by loading in the Python libraries I'll be using!

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

We need some data before we can get started with neural networks, let's load it!

In [2]:
# load data
fname = 'data/fracture_data.csv'
data = pd.read_csv(fname)

data.head()

,Sample,Well,Depth,DT,RHO,PHIN,R_XO,R_LLD,R_LLS,R_DS,IL
0,1,An1,3065.13,0.5557,0.2516,0.8795,0.3548,0.6857,0.6688,0.0169,1
1,2,An1,3089.68,0.9908,0.0110,0.8999,0.6792,0.5421,0.4071,0.1350,1
2,3,An1,3098.21,0.4444,0.1961,0.5211,0.7160,0.7304,0.6879,0.0425,1
3,4,An1,3102.33,0.4028,0.3506,0.5875,0.6218,0.6127,0.5840,0.0287,1
4,5,An1,3173.25,0.3995,0.3853,0.0845,0.5074,0.8920,0.8410,0.0510,1


### Training and Testing Sets  

In order to evaluate my final model I'm going to split the data into training and testing data subsets. I'll use samples 1-29 as training data to build the neural network and evaluate the neural networks performance on samples 30-33 (as Shi did).

In [3]:
# split data frame into features (X) and labels (y)
X = data.loc[:, 'DT':'R_DS'].values
y = to_categorical((data.loc[:, 'IL'].values == 1).astype(np.int_), 2)

# split into training and testing subsets
X2, y2 = X[0:28], y[0:28]
test_X, test_y = X[29:32], y[29:32]

# get randomized datasets for training and validation
train_X, valid_X, train_y, valid_y = train_test_split(X2, y2, test_size=0.15, random_state=42)

### Building the network  

[TFLearn](tflearn.org) lets you build the network by [defining the layers](tflearn.org/layers/core). The TFLearn package does a lot of the heavy lifting for us - so I don't have to code weight initialization, forward propogating values through the network, or backward propogation of errors through the network. I can simply focus on the architecture of the network! Time to write a function to build my network.

In [4]:
# building the network
def build_model():
    
    # reset all parameters and variables
    tf.reset_default_graph()
    
    # we have 7 input logs
    net = tflearn.input_data([None, 7])
    
    # hidden layers
    net = tflearn.fully_connected(net, 15, activation='ReLU')
    
    # output layer
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.25, loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

# build and fit the model
model = build_model()
model.fit(train_X, train_y, validation_set=0.15, show_metric=True, n_epoch=1000)

---------------------------------
Run id: WMZTT0
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 19
Validation samples: 4
--
Training Step: 1  | time: 1.172s
| SGD | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 0.69424 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 2  | total loss: 0.62375 | time: 1.022s
| SGD | epoch: 002 | loss: 0.62375 - acc: 0.4737 | val_loss: 0.69696 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 3  | total loss: 0.66715 | time: 1.031s
| SGD | epoch: 003 | loss: 0.66715 - acc: 0.6459 | val_loss: 0.70069 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 4  | total loss: 0.66508 | time: 1.019s
| SGD | epoch: 004 | loss: 0.66508 - acc: 0.6746 | val_loss: 0.70500 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 5  | total loss: 0.65803 | time: 1.019s
| SGD | epoch: 005 | loss: 0.65803 - acc: 0.6813 | val_loss: 0.70956 - val_a

Training Step: 51  | total loss: 0.59936 | time: 1.014s
| SGD | epoch: 051 | loss: 0.59936 - acc: 0.6842 | val_loss: 0.74647 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 52  | total loss: 0.59764 | time: 1.018s
| SGD | epoch: 052 | loss: 0.59764 - acc: 0.6842 | val_loss: 0.74439 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 53  | total loss: 0.59582 | time: 1.013s
| SGD | epoch: 053 | loss: 0.59582 - acc: 0.6842 | val_loss: 0.74277 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 54  | total loss: 0.59390 | time: 1.023s
| SGD | epoch: 054 | loss: 0.59390 - acc: 0.6842 | val_loss: 0.74093 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 55  | total loss: 0.59188 | time: 1.022s
| SGD | epoch: 055 | loss: 0.59188 - acc: 0.6842 | val_loss: 0.73884 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 56  | total loss: 0.58976 | time: 1.014s
| SGD | epoch: 056 | loss: 0.58976 - acc: 0.6842 | val_loss: 0.73652 - val_acc: 0.5000 -- iter: 19/19
--
Training Step: 57  | total loss: 0

Training Step: 102  | total loss: 0.39336 | time: 1.013s
| SGD | epoch: 102 | loss: 0.39336 - acc: 0.8041 | val_loss: 0.46782 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 103  | total loss: 0.38803 | time: 1.014s
| SGD | epoch: 103 | loss: 0.38803 - acc: 0.8079 | val_loss: 0.46009 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 104  | total loss: 0.38273 | time: 1.014s
| SGD | epoch: 104 | loss: 0.38273 - acc: 0.8113 | val_loss: 0.45241 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 105  | total loss: 0.37746 | time: 1.015s
| SGD | epoch: 105 | loss: 0.37746 - acc: 0.8144 | val_loss: 0.44478 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 106  | total loss: 0.37222 | time: 1.014s
| SGD | epoch: 106 | loss: 0.37222 - acc: 0.8172 | val_loss: 0.43722 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 107  | total loss: 0.36702 | time: 1.014s
| SGD | epoch: 107 | loss: 0.36702 - acc: 0.8249 | val_loss: 0.42972 - val_acc: 0.7500 -- iter: 19/19
--
Training Step: 108  | total 

Training Step: 153  | total loss: 0.19080 | time: 1.030s
| SGD | epoch: 153 | loss: 0.19080 - acc: 0.9463 | val_loss: 0.17950 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 154  | total loss: 0.18828 | time: 1.014s
| SGD | epoch: 154 | loss: 0.18828 - acc: 0.9464 | val_loss: 0.17611 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 155  | total loss: 0.18581 | time: 1.026s
| SGD | epoch: 155 | loss: 0.18581 - acc: 0.9465 | val_loss: 0.17278 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 156  | total loss: 0.18338 | time: 1.016s
| SGD | epoch: 156 | loss: 0.18338 - acc: 0.9466 | val_loss: 0.16953 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 157  | total loss: 0.18099 | time: 1.028s
| SGD | epoch: 157 | loss: 0.18099 - acc: 0.9467 | val_loss: 0.16635 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 158  | total loss: 0.17865 | time: 1.018s
| SGD | epoch: 158 | loss: 0.17865 - acc: 0.9467 | val_loss: 0.16324 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 159  | total 

Training Step: 204  | total loss: 0.10419 | time: 1.028s
| SGD | epoch: 204 | loss: 0.10419 - acc: 0.9972 | val_loss: 0.07541 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 205  | total loss: 0.10312 | time: 1.030s
| SGD | epoch: 205 | loss: 0.10312 - acc: 0.9975 | val_loss: 0.07433 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 206  | total loss: 0.10206 | time: 1.028s
| SGD | epoch: 206 | loss: 0.10206 - acc: 0.9978 | val_loss: 0.07328 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 207  | total loss: 0.10101 | time: 1.015s
| SGD | epoch: 207 | loss: 0.10101 - acc: 0.9980 | val_loss: 0.07225 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 208  | total loss: 0.09999 | time: 1.025s
| SGD | epoch: 208 | loss: 0.09999 - acc: 0.9982 | val_loss: 0.07124 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 209  | total loss: 0.09898 | time: 1.016s
| SGD | epoch: 209 | loss: 0.09898 - acc: 0.9984 | val_loss: 0.07025 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 210  | total 

...

Training Step: 765  | total loss: 0.00795 | time: 1.029s
| SGD | epoch: 765 | loss: 0.00795 - acc: 1.0000 | val_loss: 0.00720 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 766  | total loss: 0.00793 | time: 1.013s
| SGD | epoch: 766 | loss: 0.00793 - acc: 1.0000 | val_loss: 0.00720 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 767  | total loss: 0.00792 | time: 1.029s
| SGD | epoch: 767 | loss: 0.00792 - acc: 1.0000 | val_loss: 0.00719 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 768  | total loss: 0.00790 | time: 1.013s
| SGD | epoch: 768 | loss: 0.00790 - acc: 1.0000 | val_loss: 0.00718 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 769  | total loss: 0.00788 | time: 1.031s
| SGD | epoch: 769 | loss: 0.00788 - acc: 1.0000 | val_loss: 0.00717 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 770  | total loss: 0.00787 | time: 1.031s
| SGD | epoch: 770 | loss: 0.00787 - acc: 1.0000 | val_loss: 0.00716 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 771  | total 

Training Step: 816  | total loss: 0.00714 | time: 1.014s
| SGD | epoch: 816 | loss: 0.00714 - acc: 1.0000 | val_loss: 0.00681 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 817  | total loss: 0.00712 | time: 1.012s
| SGD | epoch: 817 | loss: 0.00712 - acc: 1.0000 | val_loss: 0.00681 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 818  | total loss: 0.00711 | time: 1.024s
| SGD | epoch: 818 | loss: 0.00711 - acc: 1.0000 | val_loss: 0.00680 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 819  | total loss: 0.00709 | time: 1.015s
| SGD | epoch: 819 | loss: 0.00709 - acc: 1.0000 | val_loss: 0.00679 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 820  | total loss: 0.00708 | time: 1.020s
| SGD | epoch: 820 | loss: 0.00708 - acc: 1.0000 | val_loss: 0.00678 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 821  | total loss: 0.00706 | time: 1.031s
| SGD | epoch: 821 | loss: 0.00706 - acc: 1.0000 | val_loss: 0.00678 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 822  | total 

Training Step: 867  | total loss: 0.00645 | time: 1.020s
| SGD | epoch: 867 | loss: 0.00645 - acc: 1.0000 | val_loss: 0.00648 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 868  | total loss: 0.00644 | time: 1.014s
| SGD | epoch: 868 | loss: 0.00644 - acc: 1.0000 | val_loss: 0.00647 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 869  | total loss: 0.00643 | time: 1.028s
| SGD | epoch: 869 | loss: 0.00643 - acc: 1.0000 | val_loss: 0.00647 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 870  | total loss: 0.00642 | time: 1.028s
| SGD | epoch: 870 | loss: 0.00642 - acc: 1.0000 | val_loss: 0.00646 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 871  | total loss: 0.00641 | time: 1.013s
| SGD | epoch: 871 | loss: 0.00641 - acc: 1.0000 | val_loss: 0.00646 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 872  | total loss: 0.00639 | time: 1.030s
| SGD | epoch: 872 | loss: 0.00639 - acc: 1.0000 | val_loss: 0.00645 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 873  | total 

Training Step: 918  | total loss: 0.00588 | time: 1.030s
| SGD | epoch: 918 | loss: 0.00588 - acc: 1.0000 | val_loss: 0.00620 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 919  | total loss: 0.00587 | time: 1.029s
| SGD | epoch: 919 | loss: 0.00587 - acc: 1.0000 | val_loss: 0.00619 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 920  | total loss: 0.00586 | time: 1.013s
| SGD | epoch: 920 | loss: 0.00586 - acc: 1.0000 | val_loss: 0.00618 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 921  | total loss: 0.00585 | time: 1.028s
| SGD | epoch: 921 | loss: 0.00585 - acc: 1.0000 | val_loss: 0.00618 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 922  | total loss: 0.00584 | time: 1.028s
| SGD | epoch: 922 | loss: 0.00584 - acc: 1.0000 | val_loss: 0.00617 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 923  | total loss: 0.00583 | time: 1.030s
| SGD | epoch: 923 | loss: 0.00583 - acc: 1.0000 | val_loss: 0.00617 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 924  | total 

Training Step: 969  | total loss: 0.00539 | time: 1.028s
| SGD | epoch: 969 | loss: 0.00539 - acc: 1.0000 | val_loss: 0.00595 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 970  | total loss: 0.00538 | time: 1.025s
| SGD | epoch: 970 | loss: 0.00538 - acc: 1.0000 | val_loss: 0.00594 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 971  | total loss: 0.00537 | time: 1.028s
| SGD | epoch: 971 | loss: 0.00537 - acc: 1.0000 | val_loss: 0.00594 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 972  | total loss: 0.00536 | time: 1.016s
| SGD | epoch: 972 | loss: 0.00536 - acc: 1.0000 | val_loss: 0.00593 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 973  | total loss: 0.00535 | time: 1.026s
| SGD | epoch: 973 | loss: 0.00535 - acc: 1.0000 | val_loss: 0.00593 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 974  | total loss: 0.00534 | time: 1.014s
| SGD | epoch: 974 | loss: 0.00534 - acc: 1.0000 | val_loss: 0.00592 - val_acc: 1.0000 -- iter: 19/19
--
Training Step: 975  | total 

### Testing the network

Now I'll apply the neural network I built above to the test data set. The test data set was not used during training so it represents an objective way to evaluate the model

In [5]:
predictions = (np.array(model.predict(test_X))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == test_y[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

Test accuracy:  1.0


Now I have successfully trained a neural network to achieve 100% accuracy on the training set, 100% accuracy on the validation set, and 100% accuracy on the test set. My model may not be perfect for all data moving forward; however, it represents the limit of what we can do with our limited data set. More data may require additional hidden layers or new approaches. NEOS is commited to continuously experimenting with and exploiting the bleeding edge of machine learning and artificial intelligence technologies to solve geoscience problems.